# My Point of View

Machine learning focuses on making accurate predictions. But it is decisions rather than predictions that affect the world. So predictions can be useful to help with decisions. But we need decision logic to turn a prediction into a decision, and this decision logic generally receives little attention.

Reinforcement learning directly addresses decision optimization. Most current RL research is "model-free," so the policy must be learned without the benefit of prior human-infused structure. I go the opposite direction. 

Specifically, I combine RL with structural equation modeling, a technique that encodes human beliefs about how a domain into a model (typically using multiple equations, describing different pieces of the domain). 

My workflow is:
1. Use structural equation modeling to create a model of the business environment. 
2. Use real data to estimate the structural model
3. Treat the estimated model as a simulation environment, and apply reinforcement learning algorithms to find an optimal the decision policy in the simulator
4. Apply that decision policy to make optimized decisions in the real business environment

Most business problems can be approximated well in far smaller observation spaces than the perceptual problems common in RL research, so these models do not suffer from the poor robustness and high sample complexity common in today's RL research.

Combining RL and structural equation modeling will generally be labor intensive, requiring a domain expert encode their knowledge for each application. But I believe it can improve decision-making in the next few years much as machine learning is improving companies' predictions today.

# Example Use Cases

I'll briefly describe two example applications. Then I'll show code to solve the first one.

### Airline Pricing
Airlines use machine learning models to help set ticket prices. A model predicts how many tickets they can sell each day for each upcoming flight for each candidate price. In addition to price, the models consider seasonablity, competitor prices, macroeconomic variables, etc. But even a perfect predictive model wouldn't guarantee efficient price setting. 

For example, consider a flight happening in 100 days which currently has 150 unsold seats. A predictive model says you can sell 1 ticket today for \\$300 or you could sell 2 tickets if you set the price at $250. Which price should you choose? 

The predictive model alone can't tell you the right choice in this type of dynamic optimization problem. The dynamic optimization is especially important in the likely circumstance that competitors consider your price when making their pricing decisions, the same way you consider their prices.

Airlines use heuristics (e.g. a timetable of how many tickets to sell at pre-specified periods before the flight, or a rule of thumb to sell up to a pre-specified demand elasticity.) These rules of thumb allow them to convert predictions into decisions. But the heuristics seem oversimple, much like the rules of thumb they previously used to predict sales at each time.

### Grocery Store Logistics
A grocery chain ran a [predictive modeling competition on Kaggle](https://www.kaggle.com/c/favorita-grocery-sales-forecasting) to improve demand forecasts. Better forecasts allow them to purchase an amount of each food that is closer to what will actually sold. However, purchasing an optimal amount from their wholesaler is still an optimization problem.

If a model says you will likely sell 1000 pounds of broccoli in the next week, how much should you purchase from your wholesaler. If you purchase exactly 1000 pounds, there is a reasonable chance your model will have underestimated. This causes a stockout, losing profitable sales and disappointing customers.  Similarly, if the model overestimates sales, some broccoli will rot. Unless you know the model to be correct every time, you face a tradeoff.  The optimal decision depends on factors like
- Markup rate on each sale
- Cost of storage
- Spoilage rate
- Value of ensuring customers find the items they want
- Ability to mark down item midweek to ensure it is all sold
- etc.

Ideally, you'd have estimates for each of these factors for each food. This would allow you to do a formal optimization. In practice, grocery store managers likely guess at how to make these tradeoffs, much as they previously guessed at how much of each food they would sell.


I won't belabor the list of examples. **But I think it's the rule rather than the exception for business decisions to depend on optimization with many moving parts.** If this sounds contentious, I believe it's because we've acquired the habit of slicing prediction problems out of their larger business contexts so we can apply machine learning.**


# Implemention Overview
The code below solves the Airline Pricing problem. 

I simulate a data generating process for how the market works rather than using real data. This makes the example something of a toy problem, but it helps illustrate the benefits of decision optimization because we can easily test our resulting pricing strategy in the original data generating environment.

## Market Set-Up
For illustrative simplicity, this example considers a market with only two airlines. We train an agent to set prices for Jetblue; the competitor, whose prices we cannot control, is called Delta. 

The environment contains two types of information:
- Information that is known to the airlines (such as the number of seats on each flight), and
- Information that is not directly observable to the airlines (such as their competitor's pricing mechanism, and the exact mechanism that determines how many tickets are sold on each day as a function of price and other factors). 

The airline builds a model to approximate or predict factors they don't directly observe. The information they don't directly observe is stored in a `CompetitiveConditions` object. When simulating the real environment, I build `CompetitiveConditions` from the true data generating process. When optimizing pricing, the airline uses a `CompetitiveConditions` objet that uses their predictive model.

`CompetitiveConditions` objects are passed to `Market` objects, which also hold all information that is known to the airline (such as the number of seats on each flight, and the number of days before a flight takes off).

`Market` is a subclass of an OpenAI gym enviornment, so we can evaluate standard reinforcement learning algorithms for price optimization. Moreover, using a common API for the "true" market and for model-based markets makes it easy to see how a pricing policy optimized from a model-based market performs in the true market.


The details of how the market's quantity setting mechanism are not critical for understanding this notebook.
*You can safely treat the quantity setting mechanism as a black box (as the airlines do).* However, it is described in the remainder of this paragraph. Some number of customers (`POTENTIAL_CUSTOMERS_PER_DAY`) come to a website each day.  The customers' average willingness to pay for a flight on that day `demand_level`. The `demand level` on any given day is chosen from a distribution `uniform(0, MAX_DEMAND_LEVEL)`.  Each airline receives a signal about `demand_level` on that day, and the signal is the `demand_level` plus some noise that is distributed `N(0, DEMAND_SIGNAL_NOISINESS)`. This demand signal might represent a prediction of demand from a model considering seasonality, macroeconomics, etc. Additionally, each customer has idiosyncratic preferences, so their willingness to pay for a ticket on any given airline is `demand_level + customer_preference` where `customer_preference` is distributed `N(0, CUSTOMER_LEVEL_RANDOMNESS)`.  The customer considers the price for each of the two airlines and purchases a ticket from the airline that gives them the highest consumer surplus (their personal willingness to pay minus for a ticket on that airline minus the cost of a ticket on that airline).  If the customer's consumer surplus for both airlines is negative, they do not buy a ticket.


# Imports and Set Up Parameters of "Real" Market

In [1]:
import altair as alt
import numpy as np
import pandas as pd

from keras_models import get_keras_model, prep_for_keras_model
from market import Market
from market_conditions import CompetitiveConditions
from model_testing import run_env
from pymc_models import WrappedPymcModel
from true_dgp import get_true_qty_demanded_fn


# Constants hidden from airlines
CUSTOMER_LEVEL_RANDOMNESS = 20
DEMAND_SIGNAL_NOISINESS = 10
MAX_DEMAND_LEVEL = 500
POTENTIAL_CUSTOMERS_PER_DAY = 20

# Constants known to airlines
SEATS_PER_FLIGHT = 250
SALES_WINDOW_LENGTH = 180


I guessed a reasonable pricing function (with some trial and error), and this example uses this as the underlying price function used to create the real data.

In [2]:
def simple_price_fn(my_demand_signal, days_before_flight, my_seats_avail, competitor_full): 
    # Charge more if you have a lot of time to sell seats, few seats are available, or you have little competition
    # On net, prices may increase over time because low seat inventory overwhelms remaining time effect.
    return 50 + my_demand_signal + 0.6 * days_before_flight - my_seats_avail + 40 * int(competitor_full)


# Run Real Market

Airlines have historical data they can use to build a model. Here, we run the "real" environment to create 200 flights worth of data.

In [3]:
real_market_conditions = CompetitiveConditions(delta_price_fn = simple_price_fn, 
                                               qty_fn=get_true_qty_demanded_fn(POTENTIAL_CUSTOMERS_PER_DAY, CUSTOMER_LEVEL_RANDOMNESS))
real_market = Market(real_market_conditions, MAX_DEMAND_LEVEL, DEMAND_SIGNAL_NOISINESS, SEATS_PER_FLIGHT, SALES_WINDOW_LENGTH) 

train_profits, train_data = run_env(real_market, simple_price_fn, n_times=500)
val_profits, val_data = run_env(real_market, simple_price_fn, n_times=100)


# Fit Model

We fit a model that predicts Delta's price and the quantity sold as a function of
- Days remaining
- Jetblue's demand signal
- Jetblue's remaining number of seats available
- Whether Delta's flight is fully booked (i.e. whether Delta is still selling tickets)

The model is available in `keras_models`.  It's nothing sophisticated.

In [4]:
train_x, train_y = prep_for_keras_model(train_data)
val_x, val_y = prep_for_keras_model(val_data)
keras_model = get_keras_model(train_x, train_y, val_x, val_y, verbose=0)

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(keras_model, to_file='model_visualization.png')
#IPython.display.Image("model_visualization.png")

# Set Up Model-Based Market Simulator

Now create a market based not on the true data generating processes (which the firms don't know), but instead using their predictive model.

Print out the predicted profits from using Jetblue's current pricing function in the training, val and simulator data.  If the model were perfect, the profits in the simulator would be extremely close to the profits in the training and validation data.

Note that there's a lot of dynamics from outside any ML model being used to estimate profits, since the firms are setting prices day-after-day.

In [ ]:
sim_market_conditions = CompetitiveConditions(keras_model=keras_model)
sim_market = Market(sim_market_conditions, MAX_DEMAND_LEVEL, DEMAND_SIGNAL_NOISINESS, SEATS_PER_FLIGHT, SALES_WINDOW_LENGTH)

simple_price_sim_profits, simple_price_sim_data = run_env(sim_market, simple_price_fn, n_times=100)

print("Mean profits in training data: {} \n"
      "Mean profits in val data: {} \n"
      "Mean profits in sim data: {} \n".format(train_profits.mean(), val_profits.mean(), simple_price_sim_profits.mean()))

Mean profits in training data: 57638.546 
Mean profits in val data: 56504.1 
Mean profits in sim data: 78880.4 



# Experiment With Different Pricing Strategies

Now that we have a simulator, we could apply an arbitrary reinforcement learning algorithm to create a better pricing policy. While we use the simulator for learning, our goal is to have a pricing policy that increases profits in the real environment.

For simplicity, we start with something simpler than an arbitrary reinfrocement learning model. That is, we have a function `pricing_fn_maker` that returns the previously used pricing function by some constant. This would conceivably correct for systematic overpricing or underpricing in the original pricing function.

In [ ]:
def pricing_fn_maker(baseline_price_multiplier):
    def new_price_fn(*args, **kwargs):
        return baseline_price_multiplier * simple_price_fn(*args, **kwargs)
    return new_price_fn

alternative_pricing_profits = []
alternative_pricing_scenario_details = {}

for price_mult in [0.5, 0.75, 1, 1.25, 1.5]:
    new_price_fn = pricing_fn_maker(price_mult)
    pred_profits, pred_data = run_env(sim_market, new_price_fn, n_times=100)
    actual_profits, actual_data = run_env(real_market, new_price_fn, n_times=100)
    alternative_pricing_profits.append((price_mult, pred_profits.mean(), actual_profits.mean()))
    alternative_pricing_scenario_details[price_mult] = {'pred': pred_data,
                                                        'actual': actual_data}

price_comparison = pd.DataFrame(alternative_pricing_profits, 
                                columns=['base_price_mult', 'mean_predicted_rev', 'mean_actual_rev'])

print(price_comparison)

alt.Chart(price_comparison).mark_point().encode(
    x='mean_predicted_rev',
    y='mean_actual_rev',
    color='base_price_mult',
    tooltip='base_price_mult')

The results vary from run-to-run (primarily due to variation in the trained model).

Across all runs, the existing pricing policy is better in the real environment than any alternatives being considered. 

Profits in the simulator increase monotonically in some runs. This happens because the model does a poor job estimating how Jetblue Price impacts total seats sold.

Here is a cross-tab of total seats sold across 100 flights vs the pricing multiplier

In [ ]:
pd.DataFrame([{'price_multiplier': i, 
               'total_seats_sold': alternative_pricing_scenario_details[i]['pred'].jetblue_seats_sold.sum()}
                      for i in alternative_pricing_scenario_details.keys()])

Below, I show predicted quantities sold in a given day for various candidate jetblue prices (holding the number of days until the flight constant at 100, and Jetblue's demand signal constant at 100.)

Whenever optimization yielded odd results, the model had reported upward sloping demand in the table below.

In [ ]:
days_before_flight = jb_demand_signal = 100
pred_outcomes_diff_jb_prices = []
for jb_price in [0, 25, 50, 100, 150, 200, 250]:
    # Some extra munging here do to messiness associated with multi-input / multi-output model.
    # each input fed in as separate array to facilitate hiding jetblue_price from prediction of delta_price
    prediction_data = prep_for_keras_model([days_before_flight, jb_demand_signal, jb_price], skip_y=True)
    prediction = keras_model.predict(prediction_data)
    delta_price, jb_seats_sold, delta_seats_sold = [i[0][0] for i in prediction]
    pred_outcomes_diff_jb_prices.append({'jb_price': jb_price,
                                         'delta_price': delta_price,
                                         'jetblue_seats_sold': jb_seats_sold,
                                         'delta_seats_sold': delta_seats_sold})
pd.DataFrame(pred_outcomes_diff_jb_prices).set_index(['jb_price'])

In [ ]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ppo2 import PPO2

env = DummyVecEnv([lambda: sim_market])  # The algorithms require a vectorized environment to run

model = PPO2(MlpPolicy, env, verbose=0)

def evaluate(model, num_steps=100):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 10 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obs, rewards, dones, info = env.step(action)
      
      # Stats
      episode_rewards[-1] += rewards[0]
      if dones[0]:
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 10 episodes
  mean_10ep_reward = round(np.mean(episode_rewards[-10:]), 1)
  print("Mean reward:", mean_10ep_reward, "Num episodes:", len(episode_rewards))
  return mean_10ep_reward

mean_reward_before_train = evaluate(model, num_steps=100)

# Train the agent for 10000 steps
model.learn(total_timesteps=10000)
mean_reward = evaluate(model, num_steps=10000)

In [ ]:
from stable_baselines.ppo2 import PPO2
# try to train in one line
model = PPO2('MlpPolicy', env, verbose=1).learn(1000)